In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
import os
root_path = 'gdrive/My Drive/EACL/'
os.chdir(root_path)

Mounted at /content/gdrive


In [2]:
!pip install sentencepiece
!pip install transformers
!pip install demoji

     |████████████████████████████████| 1.2MB 11.6MB/s 
     |████████████████████████████████| 1.8MB 20.0MB/s 
     |████████████████████████████████| 2.9MB 44.6MB/s 
     |████████████████████████████████| 890kB 45.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=2e7031832512c647f9b5c46c29791026502a214ae1eb7599c944c1b645ca2e10
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!pip install git+git://github.com/irshadbhat/indic-trans.git

  Cloning git://github.com/irshadbhat/indic-trans.git to /tmp/pip-req-build-mirxdpxj
  Running command git clone -q git://github.com/irshadbhat/indic-trans.git /tmp/pip-req-build-mirxdpxj
ERROR: Operation cancelled by user


In [4]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import copy
from transformers import BertModel, RobertaModel, BertTokenizer, RobertaTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, random_split, DataLoader, IterableDataset, ConcatDataset
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score 
from tqdm import tqdm
import demoji 
# from indictrans import Transliterator
demoji.download_codes() 
plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams.update({'font.size': 8})
RANDOM_SEED = 42
path = Path('./')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

... OK (Got response in 0.13 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [5]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.61', '1.7.0+cu101')

In [6]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
random_seed(RANDOM_SEED, True)

In [7]:
def handle_all_caps(t: str) -> str:
    tokens = t.split()
    tokens = replace_all_caps(tokens)
    return ' '.join(tokens)

def handle_upper_case_first_letter(t: str) -> str:
    tokens = t.split()
    tokens = deal_caps(tokens)
    return ' '.join(tokens)

def lower_case_everything(t: str) -> str:
    return t.lower().replace('@user', '').replace('#tag ', '').replace('rt ', '').strip()

In [8]:
class CodeMixedMalTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str(path/"./Mal/mlen_spm.model"))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

sp = spm.SentencePieceProcessor()
sp.Load(str(path/"./Mal/mlen_spm.model"))
itos = [sp.IdToPiece(int(i)) for i in range(25000)]
mlen_vocab = Vocab(itos)

In [ ]:
itos[:20]

['xxunk',
 'xxbos',
 'xxeos',
 'xxpad',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '.',
 ',',
 'ൽ',
 '▁the',
 'ർ',
 '▁',
 'ൻ',
 's',
 '▁•',
 '▁of',
 'ൾ']

In [9]:
class Dataset():
    def __init__(self, train_data, val_data, test_data):
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data
        # self.batch_size = batch_size

        self.label_dict = {}
        
        self.count_dic = {}

        # self.trn = Transliterator(source='tam', target='eng', build_lookup=True)

        self.train_df = pd.DataFrame(self.process_data(self.train_data))
        self.val_df = pd.DataFrame(self.process_data(self.val_data))
        self.test_df = pd.DataFrame(self.process_data(self.test_data, test = True))

    def is_english(self, s: str) -> bool:
        if len(re.findall(u'[\u0900-\u097F]', s)) <= 1000:
            return True
        return False

    def process_data(self, data, test = False):
        sentences, labels = [], []
        print(len(data))
        for line in data:
            sentence = line.strip().split('\t')
            if not test:
                label = sentence.pop()
                if label not in self.label_dict:
                    self.label_dict[label] = len(self.label_dict)
                labels.append(self.label_dict[label])
                self.count_dic[label] = self.count_dic.get(label, 0) + 1
            sentence = ' '.join(sentence)
            emoji_dict = demoji.findall(sentence)

            # if not self.is_english(sentence):
            #     print(sentence)
            #     sentence = self.trn.transform(sentence)
            #     print(sentence)

            # sentence = self.trn.transform(sentence)

            if len(emoji_dict): 
                for emoji, text in emoji_dict.items():
                    sentence = sentence.replace(emoji, ' '+text+' ')
                    sentence = ' '.join(sentence.split())
            
            sentences.append(sentence)
            
            # labels.append(label)
            
        if test: return {'input': sentences}
        return {'input': sentences, 'label': (labels)}

In [10]:
with open('Dataset/mal_full_offensive_train.csv', 'r') as f:
    train_data = f.readlines()
with open('Dataset/mal_full_offensive_dev.csv', 'r') as f:
    val_data = f.readlines()
with open('Dataset/mal_full_offensive_test.csv', 'r') as f:
    test_data = f.readlines()
data = Dataset(train_data, val_data, test_data)

16010
1999
2001


In [11]:
data.test_df.head()

,input
0,അപ്പൊ ഇതൊരൊന്നൊരാ മൊതലാണല്ലേ Suraj ആണ് നടൻ ന്...
1,എന്ത് ഊള എഡിറ്റിംഗ് ആടോ ഇത് ഒരുമാതിരി vivo vid...
2,Fefka ee padam release cheyyan samadhicho?
3,അആഹാ.. സംഗീതം ജെക്‌സ് ബിജോയ് ആണ് അപ്പൊ പൊട്ടലു...
4,Ravile thane views likes ethra ayyi enn nokan ...


In [12]:
from collections import Counter
Counter(data.train_df['label'])

Counter({0: 14153, 1: 239, 2: 1287, 3: 140, 4: 191})

In [13]:
label_cols = ['label']
text_cols = ['input']

In [14]:
tokenizer = Tokenizer(lang='mlen', tok_func=CodeMixedMalTokenizer)
tokenizer.pre_rules.append(lower_case_everything)
tokenizer.pre_rules.append(handle_all_caps)
tokenizer.pre_rules.append(handle_upper_case_first_letter)

In [15]:
tokenizer.special_cases, tokenizer.pre_rules, tokenizer.post_rules

(['xxunk',
  'xxpad',
  'xxbos',
  'xxeos',
  'xxfld',
  'xxmaj',
  'xxup',
  'xxrep',
  'xxwrep'],
 [<function fastai.text.transform.fix_html>,
  <function fastai.text.transform.replace_rep>,
  <function fastai.text.transform.replace_wrep>,
  <function fastai.text.transform.spec_add_spaces>,
  <function fastai.text.transform.rm_useless_spaces>,
  <function __main__.lower_case_everything>,
  <function __main__.handle_all_caps>,
  <function __main__.handle_upper_case_first_letter>],
 [<function fastai.text.transform.replace_all_caps>,
  <function fastai.text.transform.deal_caps>])

In [16]:
tokens = tokenizer.process_all(['Tell me about TOUR self, mujhe jaanna hai'])
''.join(tokens[0])

'▁tell▁me▁about▁tour▁self,▁mujhe▁jaanna▁hai'

In [17]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=data.train_df, valid_df=data.val_df, test_df=data.test_df, tokenizer=tokenizer, vocab=mlen_vocab, label_cols=label_cols, text_cols=text_cols)

/usr/local/lib/python3.6/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


In [ ]:
data_lm.show_batch()

/usr/local/lib/python3.6/dist-packages/fastai/text/data.py:339: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  idx_min = (t != self.pad_idx).nonzero().min()


idx,text
0,▁മീ ശ ▁പ ിരിക്കുന്ന ▁ആര ണ്ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁royal ▁me ch ▁ആ ടാ ▁x x bo s ▁su shi n ▁s yam ▁sha i ju ▁khalid ▁mid hu n ▁manu al ▁x x bo s ▁j ▁a ▁k ▁e ▁s . ▁b ▁ej ▁o ▁ y ▁ xxunk ▁x x bo s ▁p w oli ▁it em ▁pada m ▁kanan da ▁ennu ▁karu thiya tha
1,▁sa ch y ▁സർ ▁ആണ് . . . ▁ഉറപ്പ ായ ിട്ടും ▁ഒരു ▁കിട ില ൻ ▁t w ist ▁ഉണ്ടാക്ക ും ▁x x bo s ▁12 : 00 ▁pi di ch ▁ninn ▁e ni ▁su ga yi tt ▁ki da nn ▁u ranga ▁e ni ▁release ▁n ▁vendi ▁i am ▁wait ing ▁x x bo s ▁v y shak ▁sir ▁trai ler ▁search ▁che y thi tt ▁ki ttu ni lla
2,k karan ▁p l z ▁support ▁x x bo s ▁കണ്ണു ം ▁പൂ ട്ടി ▁പറയാം . . ▁ഇതൊരു ▁വൻ പ രാജ യം ▁ആയിരിക്കും ▁x x bo s ▁wa wu . . vari e ty ▁a yi tt ▁in d . . ▁oru ▁korea n ▁movie ▁level . . ▁x x bo s ▁നല്ല ▁നല്ല ▁എഡിറ്റ ിംഗ് ▁ഫോട്ടോ സിന് ▁എന്റെ ▁ചാനൽ ▁സബ് സ് ക്രൈ ബ് ▁ചെ യ ുക ▁x x bo
3,▁x x bo s ▁man ju ▁che chi ▁oru ▁25 ▁vay a su ▁ kash th ichu ▁tho nn i kum ▁enna ▁sun dhari ▁a anu u u ▁x x bo s ▁കാണ ാട ▁ഹാ ▁കാണാം ▁de c ▁20 ▁x x bo s ▁അവൾ ▁ഒറ്റ ക്കാണ് ▁വന്നത് ▁mon es tar ▁man ju ▁ചേ ച്ചി ▁uy ir ▁x x bo s ▁ഇതിലെ ▁ഒരു ▁പ്ര േ ത് യേ ക ത ▁എന്ത ാണ്
4,s ▁വേട്ട ക്ക് ▁ശേഷം ▁അടുത്ത ▁ത്രി ല്ല െ ർ ▁i am ▁wait ing ▁x x bo s ▁la le ttan ▁aka ▁it ty maan i ▁- ▁super ▁du per ▁fan ta stic ▁bomb a stic ▁super la stic . . . ▁x x bo s ▁t g f nu m ▁masterpiece nu m ▁shesha m ▁adut ha ▁v van ▁hit ▁aa kan ▁po kkunn a ▁film ▁x x bo s


In [ ]:
# data_lm.vocab.itos = data_lm.vocab.itos
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, pretrained=False, model_dir = './Mal/models/models')

In [ ]:
learn.load('best_model',with_opt=True, device=device)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (16010 items)
x: LMTextList
▁x x bo s ▁പല ദേശ ം . ▁പല ▁ഭാഷ ▁ഒരേ ▁ഒരു ▁രാജാവ് ▁അല്ലാതെ ▁സ്വന്തം ▁രാജ വയ ത് ▁അല്ല,▁x x bo s ▁ഈ ▁ഓണ ം ▁ഏ ട്ട നും ▁പിള്ള േ ർക്ക് ▁ഉള്ള താണ് ▁എന്ന് ▁ഉള്ള വർ ▁ലൈ ക് ▁അടി,▁x x bo s ▁ആര ണ്ട ▁ആര ണ്ട ▁തല ുണ്ടാക ാ ണാ ▁ആര ണ്ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁royal ▁me ch ▁ആ ടാ ▁ആര ണ്ട ▁ആര ണ്ട ▁മീ ശ ▁പ ിരിക്കുന്ന ▁ആര ണ്ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁royal ▁me ch ▁ആ ടാ,▁x x bo s ▁su shi n ▁s yam ▁sha i ju ▁khalid ▁mid hu n ▁manu al,▁x x bo s ▁j ▁a ▁k ▁e ▁s . ▁b ▁ej ▁o ▁ y ▁ xxunk
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (1999 items)
x: LMTextList
▁x x bo s ▁gopi che tta nte ▁b g m ▁ um ▁mam mo o kayu m ▁ishta pedunn avar ▁like ▁ xxunk,▁x x bo s ▁ഇത് ▁ഒരു ▁പെണ്ണ ് ▁തന്നെ ▁ആ ണോ ▁direct ▁ചെയ്യുന്നത് ▁poli chu ▁ni vin,▁x x bo s ▁പൃഥ്വി രാജ് ▁സു രാജ േ ട്ട ൻ ▁ലാ ലും ▁അല ക്സ് . . ▁lal ▁j r . ▁ന്റെ ▁ഒരു ▁വെ റൈ റ്റി ▁ചിത്രം ▁പ്രതീക്ഷിക്ക ുന്നു . . ▁best ▁wi shes ▁team,▁x x bo s ▁പോക രുത് ▁മക്കള െ ▁പോ ക്ക ▁ xxrep ▁10 ▁

In [ ]:
learn.freeze()
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.743490,4.369740,0.316741,00:23


In [ ]:
learn.save('fit_head', with_opt=True)

In [ ]:
learn.load('fit_head', with_opt=True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (16010 items)
x: LMTextList
▁x x bo s ▁പല ദേശ ം . ▁പല ▁ഭാഷ ▁ഒരേ ▁ഒരു ▁രാജാവ് ▁അല്ലാതെ ▁സ്വന്തം ▁രാജ വയ ത് ▁അല്ല,▁x x bo s ▁ഈ ▁ഓണ ം ▁ഏ ട്ട നും ▁പിള്ള േ ർക്ക് ▁ഉള്ള താണ് ▁എന്ന് ▁ഉള്ള വർ ▁ലൈ ക് ▁അടി,▁x x bo s ▁ആര ണ്ട ▁ആര ണ്ട ▁തല ുണ്ടാക ാ ണാ ▁ആര ണ്ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁royal ▁me ch ▁ആ ടാ ▁ആര ണ്ട ▁ആര ണ്ട ▁മീ ശ ▁പ ിരിക്കുന്ന ▁ആര ണ്ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁royal ▁me ch ▁ആ ടാ,▁x x bo s ▁su shi n ▁s yam ▁sha i ju ▁khalid ▁mid hu n ▁manu al,▁x x bo s ▁j ▁a ▁k ▁e ▁s . ▁b ▁ej ▁o ▁ y ▁ xxunk
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (1999 items)
x: LMTextList
▁x x bo s ▁gopi che tta nte ▁b g m ▁ um ▁mam mo o kayu m ▁ishta pedunn avar ▁like ▁ xxunk,▁x x bo s ▁ഇത് ▁ഒരു ▁പെണ്ണ ് ▁തന്നെ ▁ആ ണോ ▁direct ▁ചെയ്യുന്നത് ▁poli chu ▁ni vin,▁x x bo s ▁പൃഥ്വി രാജ് ▁സു രാജ േ ട്ട ൻ ▁ലാ ലും ▁അല ക്സ് . . ▁lal ▁j r . ▁ന്റെ ▁ഒരു ▁വെ റൈ റ്റി ▁ചിത്രം ▁പ്രതീക്ഷിക്ക ുന്നു . . ▁best ▁wi shes ▁team,▁x x bo s ▁പോക രുത് ▁മക്കള െ ▁പോ ക്ക ▁ xxrep ▁10 ▁

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(7, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.224651,4.195209,0.335733,00:29
1,4.001050,3.928734,0.367615,00:30
2,3.725434,3.754316,0.386217,00:29
3,3.520085,3.646782,0.399795,00:30
4,3.341885,3.591532,0.406455,00:29
5,3.255836,3.571074,0.408854,00:29
6,3.207998,3.568316,0.409226,00:30


In [ ]:
learn.save('fine_tuned', with_opt=True)

In [ ]:
learn.load('fine_tuned', with_opt=True);
learn.predict('en payar',n_words=10)

'en payar ▁a ayirik kum ▁anu nu ▁pakar i ▁patti lla til'

In [ ]:
learn.save_encoder('fine_tuned_enc')

In [ ]:
# data.train_df['ohe'] = data.train_df['label'].apply(change_1h_targ)
# data.val_df['ohe'] = data.val_df['label'].apply(change_1h_targ)
label_cols = ['label']
text_cols = ['input']

In [ ]:
data.train_df

,input,label
0,പലദേശം. പല ഭാഷ ഒരേ ഒരു രാജാവ് അല്ലാതെ സ്വന്ത...,0
1,ഈ ഓണം ഏട്ടനും പിള്ളേർക്ക് ഉള്ളതാണ് എന്ന് ഉള്ളവ...,0
2,ആരണ്ട ആരണ്ട തലുണ്ടാകാണാ ആരണ്ട ഞാൻ ആണ്ട ഞാൻ ആണ്...,0
3,Sushin syam Shaiju khalid Midhun manual,0
4,J A K E S. B EJ O Y !!!,0
...,...,...
16005,കട്ട ലാലേട്ടൻ ഫാൻസിന് ദൈവത്തെ ഓർത്ത് അമിത പ്രത...,0
16006,ente mammookka ningal puliyalla oru simhama......,0
16007,Lucifer mass dialogues Ellam onnu comment chey...,0
16008,Like from Madurai (Tamil nadu) ....,2


In [18]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=data.train_df, valid_df=data.val_df, test_df = data.test_df, tokenizer=tokenizer, vocab=mlen_vocab, bs=16, label_cols = label_cols, text_cols=text_cols)
data_clas.show_batch()

/usr/local/lib/python3.6/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/fastai/text/data.py:339: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  idx_min = (t != self.pad_idx).nonzero().min()


text,target
▁x x bo s ▁മന മറിയ ുന്ന ോ ള ് ▁ xxunk ▁r ▁ഇവ ളാ ▁കൊട്ട ് യോ ള ് ▁ xxunk ▁r ▁കന വില ് ▁വന്ന ോ ള ് ▁ xxunk ▁r ▁നി ന് ▁കര ള ായ് ▁ പോന്ന ോ ള ് ▁ xxunk ▁r ▁പരി ണ യ രാ വില ് ▁പ വ നു ര ുക ും ▁പോ ല് ▁ xxunk ▁r ▁ഹൃദയ ം ▁തന്ന ോ ള ് ▁,0
▁x x bo s ▁ കാത്ത ിരിക്കുന്നു ▁ കാത്ത ിരിക്കുന്നു . . ▁ കാത്ത ിരിക്കുന്നു ▁ കാത്ത ിരിക്കുന്നു . . ▁ കാത്ത ിരിക്കുന്നു ▁ കാത്ത ിരിക്കുന്നു . . ▁ കാത്ത ിരിക്കുന്നു ▁ കാത്ത ിരിക്കുന്നു . . ▁ കാത്ത ിരിക്കുന്നു ▁ കാത്ത ിരിക്കുന്നു . . ▁ കാത്ത ിരിക്കുന്നു ▁ കാത്ത ിരിക്കുന്നു . . ▁ കാത്ത ിരിക്കുന്നു ▁ കാത്ത ിരിക്കുന്നു . . ▁ കാത്ത ിരിക്കുന്നു ▁ കാത്ത ിരിക്കുന്നു . . ▁ കാത്ത,0
▁x x bo s ▁priya pe tta ▁ji bi ▁and ▁jo ju ▁ivi de ▁kure ▁pala than tha kku pi ran na ▁kure ▁la le ttan ▁ha ter s ▁e ▁nall a ▁cinema ye ▁de grade ▁cheyya n ▁shram ikkunn und ▁pakshe ▁enik ku ra ppan u ▁ni ng alu de ▁e ▁kanni ▁cinema ye ▁malayali pre k sha kar ▁iru ▁kay yu m ▁ne e tti ▁swe e kari,4
▁x x bo s ▁trai ler ▁kand ittu ▁ivi de ▁ninnu m ▁po kan ▁pattu nn illa . ▁ve end um ▁ve end um ▁vannu ▁kanunnu . ▁oru ▁karyam ▁ura pp ▁man ju ▁che chi yu de ▁thir ichu vara v ile ▁etta vu m ▁mika cha ▁ka dha path ram ▁a th ▁madhuri ▁than ne ▁a ayirik kum . ▁ asura nuk kum ▁me le . . ▁pha rs,0
▁x x bo s ▁it ho ke ▁ kanu bo ▁mana sil ▁oru ▁san tho sham . . ▁feel ▁good ▁math ram ▁kiti ▁kond irunn a ▁malaya m ▁ippo ▁mari ▁. . ▁malayalam ▁industry ▁cinema yu de ▁mat tu ▁tha lang ile k ▁a thi pet u ▁kond irik u nu . . ▁hu gg ing ▁face ▁hu gg ing ▁face ▁. . ▁je lli ▁kettu ▁pole ▁moo th on,0


In [19]:
data_clas.sanity_check()

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [20]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, model_dir = './Mal/models/models')
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (16010 items)
x: TextList
▁x x bo s ▁പല ദേശ ം . ▁പല ▁ഭാഷ ▁ഒരേ ▁ഒരു ▁രാജാവ് ▁അല്ലാതെ ▁സ്വന്തം ▁രാജ വയ ത് ▁അല്ല,▁x x bo s ▁ഈ ▁ഓണ ം ▁ഏ ട്ട നും ▁പിള്ള േ ർക്ക് ▁ഉള്ള താണ് ▁എന്ന് ▁ഉള്ള വർ ▁ലൈ ക് ▁അടി,▁x x bo s ▁ആര ണ്ട ▁ആര ണ്ട ▁തല ുണ്ടാക ാ ണാ ▁ആര ണ്ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁royal ▁me ch ▁ആ ടാ ▁ആര ണ്ട ▁ആര ണ്ട ▁മീ ശ ▁പ ിരിക്കുന്ന ▁ആര ണ്ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁royal ▁me ch ▁ആ ടാ,▁x x bo s ▁su shi n ▁s yam ▁sha i ju ▁khalid ▁mid hu n ▁manu al,▁x x bo s ▁j ▁a ▁k ▁e ▁s . ▁b ▁ej ▁o ▁ y ▁ xxunk
y: CategoryList
0,0,0,0,0
Path: .;

Valid: LabelList (1999 items)
x: TextList
▁x x bo s ▁gopi che tta nte ▁b g m ▁ um ▁mam mo o kayu m ▁ishta pedunn avar ▁like ▁ xxunk,▁x x bo s ▁ഇത് ▁ഒരു ▁പെണ്ണ ് ▁തന്നെ ▁ആ ണോ ▁direct ▁ചെയ്യുന്നത് ▁poli chu ▁ni vin,▁x x bo s ▁പൃഥ്വി രാജ് ▁സു രാജ േ ട്ട ൻ ▁ലാ ലും ▁അല ക്സ് . . ▁lal ▁j r . ▁ന്റെ ▁ഒരു ▁വെ റൈ റ്റി ▁ചിത്രം ▁പ്രതീക്ഷിക്ക ുന്നു . . ▁best ▁wi shes ▁team,▁x x bo s ▁പോക രുത് ▁മക്കള െ ▁പോ ക്ക ▁ xxrep ▁10 ▁.

In [ ]:
learn.freeze()
learn.loss_func.func

CrossEntropyLoss()

In [ ]:
f1 = FBeta(beta = 1,average = 'weighted')

In [ ]:
learn.metrics = [accuracy, f1]

In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,f_beta,time
0,0.261469,0.239057,0.925463,0.915557,00:21


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learn.save('first-full')

In [ ]:
learn.load('first-full')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (16010 items)
x: TextList
▁x x bo s ▁പല ദേശ ം . ▁പല ▁ഭാഷ ▁ഒരേ ▁ഒരു ▁രാജാവ് ▁അല്ലാതെ ▁സ്വന്തം ▁രാജ വയ ത് ▁അല്ല,▁x x bo s ▁ഈ ▁ഓണ ം ▁ഏ ട്ട നും ▁പിള്ള േ ർക്ക് ▁ഉള്ള താണ് ▁എന്ന് ▁ഉള്ള വർ ▁ലൈ ക് ▁അടി,▁x x bo s ▁ആര ണ്ട ▁ആര ണ്ട ▁തല ുണ്ടാക ാ ണാ ▁ആര ണ്ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁royal ▁me ch ▁ആ ടാ ▁ആര ണ്ട ▁ആര ണ്ട ▁മീ ശ ▁പ ിരിക്കുന്ന ▁ആര ണ്ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁royal ▁me ch ▁ആ ടാ,▁x x bo s ▁su shi n ▁s yam ▁sha i ju ▁khalid ▁mid hu n ▁manu al,▁x x bo s ▁j ▁a ▁k ▁e ▁s . ▁b ▁ej ▁o ▁ y ▁ xxunk
y: CategoryList
0,0,0,0,0
Path: .;

Valid: LabelList (1999 items)
x: TextList
▁x x bo s ▁gopi che tta nte ▁b g m ▁ um ▁mam mo o kayu m ▁ishta pedunn avar ▁like ▁ xxunk,▁x x bo s ▁ഇത് ▁ഒരു ▁പെണ്ണ ് ▁തന്നെ ▁ആ ണോ ▁direct ▁ചെയ്യുന്നത് ▁poli chu ▁ni vin,▁x x bo s ▁പൃഥ്വി രാജ് ▁സു രാജ േ ട്ട ൻ ▁ലാ ലും ▁അല ക്സ് . . ▁lal ▁j r . ▁ന്റെ ▁ഒരു ▁വെ റൈ റ്റി ▁ചിത്രം ▁പ്രതീക്ഷിക്ക ുന്നു . . ▁best ▁wi shes ▁team,▁x x bo s ▁പോക രുത് ▁മക്കള െ ▁പോ ക്ക ▁ xxrep ▁10 ▁.

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,f_beta,time
0,0.276670,0.218314,0.925463,0.920322,00:24


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learn.save('second-full')

In [ ]:
learn.load('second-full')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (16010 items)
x: TextList
▁x x bo s ▁പല ദേശ ം . ▁പല ▁ഭാഷ ▁ഒരേ ▁ഒരു ▁രാജാവ് ▁അല്ലാതെ ▁സ്വന്തം ▁രാജ വയ ത് ▁അല്ല,▁x x bo s ▁ഈ ▁ഓണ ം ▁ഏ ട്ട നും ▁പിള്ള േ ർക്ക് ▁ഉള്ള താണ് ▁എന്ന് ▁ഉള്ള വർ ▁ലൈ ക് ▁അടി,▁x x bo s ▁ആര ണ്ട ▁ആര ണ്ട ▁തല ുണ്ടാക ാ ണാ ▁ആര ണ്ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁royal ▁me ch ▁ആ ടാ ▁ആര ണ്ട ▁ആര ണ്ട ▁മീ ശ ▁പ ിരിക്കുന്ന ▁ആര ണ്ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁royal ▁me ch ▁ആ ടാ,▁x x bo s ▁su shi n ▁s yam ▁sha i ju ▁khalid ▁mid hu n ▁manu al,▁x x bo s ▁j ▁a ▁k ▁e ▁s . ▁b ▁ej ▁o ▁ y ▁ xxunk
y: CategoryList
0,0,0,0,0
Path: .;

Valid: LabelList (1999 items)
x: TextList
▁x x bo s ▁gopi che tta nte ▁b g m ▁ um ▁mam mo o kayu m ▁ishta pedunn avar ▁like ▁ xxunk,▁x x bo s ▁ഇത് ▁ഒരു ▁പെണ്ണ ് ▁തന്നെ ▁ആ ണോ ▁direct ▁ചെയ്യുന്നത് ▁poli chu ▁ni vin,▁x x bo s ▁പൃഥ്വി രാജ് ▁സു രാജ േ ട്ട ൻ ▁ലാ ലും ▁അല ക്സ് . . ▁lal ▁j r . ▁ന്റെ ▁ഒരു ▁വെ റൈ റ്റി ▁ചിത്രം ▁പ്രതീക്ഷിക്ക ുന്നു . . ▁best ▁wi shes ▁team,▁x x bo s ▁പോക രുത് ▁മക്കള െ ▁പോ ക്ക ▁ xxrep ▁10 ▁.

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='f_beta', name='final')])

epoch,train_loss,valid_loss,accuracy,f_beta,time
0,0.198280,0.213743,0.927464,0.926764,00:50
1,0.231956,0.196560,0.937469,0.936919,00:55
2,0.140154,0.168566,0.948474,0.946639,00:53
3,0.147491,0.162028,0.947974,0.948663,00:55
4,0.094700,0.165313,0.953977,0.953641,00:52
5,0.079394,0.191512,0.946473,0.948664,00:54
6,0.068090,0.157436,0.961481,0.960253,00:56
7,0.042953,0.164013,0.963482,0.961990,00:56
8,0.038717,0.173934,0.963982,0.963085,00:56
9,0.027455,0.175998,0.957979,0.957219,00:51


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Better model found at epoch 0 with f_beta value: 0.926764190196991.


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Better model found at epoch 1 with f_beta value: 0.9369188547134399.


...

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Better model found at epoch 7 with f_beta value: 0.9619898200035095.


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Better model found at epoch 8 with f_beta value: 0.9630845189094543.


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [21]:
learn.load('final')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (16010 items)
x: TextList
▁x x bo s ▁പല ദേശ ം . ▁പല ▁ഭാഷ ▁ഒരേ ▁ഒരു ▁രാജാവ് ▁അല്ലാതെ ▁സ്വന്തം ▁രാജ വയ ത് ▁അല്ല,▁x x bo s ▁ഈ ▁ഓണ ം ▁ഏ ട്ട നും ▁പിള്ള േ ർക്ക് ▁ഉള്ള താണ് ▁എന്ന് ▁ഉള്ള വർ ▁ലൈ ക് ▁അടി,▁x x bo s ▁ആര ണ്ട ▁ആര ണ്ട ▁തല ുണ്ടാക ാ ണാ ▁ആര ണ്ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁royal ▁me ch ▁ആ ടാ ▁ആര ണ്ട ▁ആര ണ്ട ▁മീ ശ ▁പ ിരിക്കുന്ന ▁ആര ണ്ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁ആണ് ട ▁ഞാൻ ▁royal ▁me ch ▁ആ ടാ,▁x x bo s ▁su shi n ▁s yam ▁sha i ju ▁khalid ▁mid hu n ▁manu al,▁x x bo s ▁j ▁a ▁k ▁e ▁s . ▁b ▁ej ▁o ▁ y ▁ xxunk
y: CategoryList
0,0,0,0,0
Path: .;

Valid: LabelList (1999 items)
x: TextList
▁x x bo s ▁gopi che tta nte ▁b g m ▁ um ▁mam mo o kayu m ▁ishta pedunn avar ▁like ▁ xxunk,▁x x bo s ▁ഇത് ▁ഒരു ▁പെണ്ണ ് ▁തന്നെ ▁ആ ണോ ▁direct ▁ചെയ്യുന്നത് ▁poli chu ▁ni vin,▁x x bo s ▁പൃഥ്വി രാജ് ▁സു രാജ േ ട്ട ൻ ▁ലാ ലും ▁അല ക്സ് . . ▁lal ▁j r . ▁ന്റെ ▁ഒരു ▁വെ റൈ റ്റി ▁ചിത്രം ▁പ്രതീക്ഷിക്ക ുന്നു . . ▁best ▁wi shes ▁team,▁x x bo s ▁പോക രുത് ▁മക്കള െ ▁പോ ക്ക ▁ xxrep ▁10 ▁.

In [31]:
def get_predicted(preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    return pred_flat

df_test = data.val_df.copy()
from sklearn.metrics import f1_score, accuracy_score
df_dict = {'query': list(df_test['input']), 'actual_label': list(df_test['label']), 'predicted_label': [0]*df_test.shape[0]}  
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Valid, ordered=True)
df_result['predicted_label'] = list(get_predicted(np.array(preds[0])))
df_result.head()

,query,actual_label,predicted_label
0,Gopichettante BGM um mammookayum ishtapedunnav...,0,0
1,ഇത് ഒരു പെണ്ണ് തന്നെ ആണോ direct ചെയ്യുന്നത് p...,0,0
2,പൃഥ്വിരാജ് സുരാജേട്ടൻ ലാലും അലക്സ്.. Lal jr. ന...,0,0
3,പോകരുത് മക്കളെ പോക്ക .......... നൻ കണ്ട് എന്റ്...,0,0
4,avan Varum ente Makan Madhura Raja....,0,0


In [24]:
f1_score(df_result['actual_label'], df_result['predicted_label'], average = 'weighted')

0.963084526715372

In [23]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.9639819909954978

In [34]:
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)

In [32]:
logits = {}
logits['val'] = preds[0]

In [35]:
logits['test'] = preds[0]

In [36]:
logits

{'test': tensor([[9.9996e-01, 3.5191e-05, 1.4365e-07, 1.0257e-07, 2.9254e-06],
         [9.9949e-01, 6.9551e-05, 6.0725e-06, 2.8586e-05, 4.0866e-04],
         [9.9737e-01, 1.9562e-03, 4.6076e-04, 1.0261e-04, 1.1077e-04],
         ...,
         [9.9906e-01, 5.2406e-05, 1.2549e-04, 4.7273e-04, 2.9276e-04],
         [9.9998e-01, 7.3564e-06, 7.8305e-08, 5.1346e-06, 6.8880e-06],
         [9.9940e-01, 4.4977e-04, 4.8819e-06, 6.2034e-05, 8.3402e-05]]),
 'val': tensor([[9.9991e-01, 8.2056e-05, 7.4294e-06, 2.2854e-07, 1.0521e-06],
         [1.0000e+00, 7.2056e-07, 5.7720e-08, 1.1636e-07, 1.9978e-07],
         [9.9999e-01, 4.7877e-06, 2.7178e-07, 3.8088e-08, 5.9020e-07],
         ...,
         [9.6041e-01, 4.4667e-03, 1.3528e-06, 4.2075e-03, 3.0917e-02],
         [9.9804e-01, 2.3449e-04, 1.0931e-03, 2.5722e-04, 3.7776e-04],
         [9.9907e-01, 5.2230e-06, 9.1669e-04, 1.1944e-06, 4.6608e-06]])}

In [40]:
with open('mal_ulmfit_preds.pkl', 'wb') as f:
    pickle.dump(logits, f)

In [41]:
with open('mal_ulmfit_preds.pkl', 'rb') as f:
    dpred = pickle.load(f)

In [42]:
dpred

{'test': tensor([[9.9996e-01, 3.5191e-05, 1.4365e-07, 1.0257e-07, 2.9254e-06],
         [9.9949e-01, 6.9551e-05, 6.0725e-06, 2.8586e-05, 4.0866e-04],
         [9.9737e-01, 1.9562e-03, 4.6076e-04, 1.0261e-04, 1.1077e-04],
         ...,
         [9.9906e-01, 5.2406e-05, 1.2549e-04, 4.7273e-04, 2.9276e-04],
         [9.9998e-01, 7.3564e-06, 7.8305e-08, 5.1346e-06, 6.8880e-06],
         [9.9940e-01, 4.4977e-04, 4.8819e-06, 6.2034e-05, 8.3402e-05]]),
 'val': tensor([[9.9991e-01, 8.2056e-05, 7.4294e-06, 2.2854e-07, 1.0521e-06],
         [1.0000e+00, 7.2056e-07, 5.7720e-08, 1.1636e-07, 1.9978e-07],
         [9.9999e-01, 4.7877e-06, 2.7178e-07, 3.8088e-08, 5.9020e-07],
         ...,
         [9.6041e-01, 4.4667e-03, 1.3528e-06, 4.2075e-03, 3.0917e-02],
         [9.9804e-01, 2.3449e-04, 1.0931e-03, 2.5722e-04, 3.7776e-04],
         [9.9907e-01, 5.2230e-06, 9.1669e-04, 1.1944e-06, 4.6608e-06]])}